In [1]:
import dal, pandas, common
dal.init_db()
common.PERIODS_IN_HOUR = 4*60
common.PERIODS_IN_DAY = common.PERIODS_IN_HOUR * 24
from scipy.stats import multivariate_normal
from sklearn import model_selection
import seaborn as sns
import matplotlib.pyplot as plt

/usr/lib64/python2.7/site-packages/sklearn/utils/fixes.py:200: UserWarning: Using `sort` instead of partition.Upgrade numpy to 1.8 for better performace on large numberof clusters
  warnings.warn('Using `sort` instead of partition.'


In [2]:
%matplotlib inline

In [71]:
def calc_prob(row, model):
    return  model.pdf((row['duration'], row['bandwidth']))

def my_log(x):
    if isinstance(x, (int, long, float, complex)):
        return common.safe_log(common.safe_log(x))
    else:
        return x

def get_script_data(data):
    bots_ips = ['10.0.0.2','10.0.0.21','10.0.0.6','10.0.0.14','10.0.0.3', '88.198.23.221', '10.0.0.20', '10.0.0.12', '10.0.0.11',  '10.0.0.23', '10.0.0.18',  '10.0.0.24', '10.0.0.105',  '10.0.0.5', '10.0.0.104']
    return data[(data['dest_ip'].isin(bots_ips) | data['src_ip'].isin(bots_ips))]

def get_human_data(data):
    bots_ips = ['10.0.0.2','10.0.0.21','10.0.0.6','10.0.0.14','10.0.0.3', '88.198.23.221', '10.0.0.20', '10.0.0.12', '10.0.0.11',  '10.0.0.23', '10.0.0.18',  '10.0.0.24', '10.0.0.105',  '10.0.0.5', '10.0.0.104']
    return data[(data['dest_ip'].isin(bots_ips) | data['src_ip'].isin(bots_ips)) == False]

def create_model(data):
    return multivariate_normal(mean = data[['sessions_bandwidths', 'sessions_durations']].mean(), cov=data[['sessions_bandwidths', 'sessions_durations']].cov())

In [10]:
all_sessions = dal.get_ended_sessions()

In [11]:
def get_hours_kpis(all_kpis):
    from collections import defaultdict

    # get sorted by timestamp from mongo

    current_ts = all_kpis[0]['timestamp'] - 5 * common.seconds_in_day
    current_hour = 9
    kpis_by_hours = defaultdict(list)

    for i in range(24):
        kpis_by_hours[current_hour] += filter(
            lambda x: current_ts > x['timestamp'] > current_ts - common.seconds_in_hour, all_kpis)
        current_ts -= common.seconds_in_hour
        current_hour -= 1
        if current_hour == -1:
            current_hour = 23
            #         all_kpis = filter(lambda x: x['timestamp'] < current_ts, all_kpis)

    return kpis_by_hours

In [77]:
df = get_hours_kpis(all_sessions)


In [78]:
for i in df:
    df[i] =  pandas.DataFrame(df[i])

In [79]:
lens =[]
for i in df:
    lens.append(len(df[i]))
lens = pandas.DataFrame(lens)

In [80]:
for i in df:
    data = pandas.DataFrame()
    data['bandwidth'] = df[i]['n_bytes']
    data['duration'] = df[i]['timestamp'] - df[i]['start_time']
    data['dest_ip'] = df[i]['dest_ip']
    data['src_ip'] = df[i]['src_ip']
    df[i] = data

In [84]:
df[10].head()

,bandwidth,duration,dest_ip,src_ip
0,151,4.160304,10.0.0.105,107.20.161.4
1,1260,4.015238,107.20.161.4,10.0.0.105
2,7790,75.203189,10.0.0.105,107.20.161.4
3,40,0.000000,10.0.0.11,185.94.180.125
4,152,9.016457,198.151.217.186,10.0.0.12


In [85]:
for i in df:
    df[i].applymap(lambda x: my_log(x))

In [ ]:
for i in df:
    

In [10]:
data['sessions_bandwidths'] = data['sessions_bandwidths'].apply(lambda x: my_log(x))
data['sessions_durations'] = data['sessions_durations'].apply(lambda x: my_log(x))

In [145]:
model = multivariate_normal(mean = data[['sessions_bandwidths', 'sessions_durations']].mean(), cov=data[['sessions_bandwidths', 'sessions_durations']].cov())

In [146]:
data['prob'] = data.apply(calc_prob, axis=1)

In [149]:
b_mean = data['sessions_bandwidths'].mean()
d_mean = data['sessions_durations'].mean()

In [24]:
data = data.drop(data[data.sessions_durations < d_mean].index)
data = data.drop(data[data.sessions_bandwidths < b_mean].index)

In [151]:
data.sort_values('prob')

,sessions_bandwidths,sessions_durations,dest_ip,src_ip,prob
153216,2.823100,-12.273782,10.0.0.9,199.16.156.11,0.000000
168041,3.090125,-10.269954,66.235.139.17,10.0.0.9,0.000000
168048,2.823100,-13.075187,10.0.0.9,151.101.0.217,0.000000
302907,2.818704,-13.441579,10.0.0.9,52.216.65.56,0.000000
139204,2.915883,-10.629858,10.0.0.9,52.48.30.62,0.000000
121466,2.660365,-12.012736,10.0.0.9,66.211.181.192,0.000000
302906,2.823100,-11.766380,10.0.0.9,54.84.157.128,0.000000
139203,2.823100,-11.213730,10.0.0.9,52.48.224.164,0.000000
302822,3.113473,-12.042898,10.0.0.9,140.211.15.244,0.000000
302820,2.764350,-12.105182,10.0.0.9,137.227.240.37,0.000000


In [72]:
sns.distplot(data['bandwidth'].apply(lambda x: my_log(x)))

KeyError: 'bandwidth'

In [206]:
data[data['src_ip'].str.match('10.0.*')]['src_ip'].value_counts()

10.0.0.104    67714
10.0.0.105    60607
10.0.0.20     59838
10.0.0.11     59598
10.0.0.18     58886
10.0.0.12     55018
10.0.0.24     47770
10.0.0.5       3829
10.0.0.3        914
10.0.0.2          9
10.0.0.21         5
10.0.0.6          3
10.0.0.14         2
Name: src_ip, dtype: int64